In [10]:
# Step 1: Install dependencies (run once)
!pip install gitpython transformers gradio --quiet

# Step 2: Clone QuixBugs repo if not already cloned
import git
import os

if not os.path.exists("Code-Refactoring-QuixBugs"):
    print("Cloning QuixBugs repo...")
    git.Repo.clone_from("https://github.com/RumbleJack56/Code-Refactoring-QuixBugs.git", "Code-Refactoring-QuixBugs")
else:
    print("Repo already cloned.")

# Step 3: Imports
import re
import importlib.util
import sys
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

# Step 4: Load model and tokenizer
print("Loading model (this may take a minute)...")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Step 5: Bug classification heuristic mapping
bug_classes = {
    "off-by-one": ["off-by-one", "index+1", "index - 1", "range(", "loop condition"],
    "incorrect condition": ["if ", "while ", "condition wrong", "incorrect condition"],
    "wrong operator": ["+", "-", "*", "/", "%", "==", "!=", "<", ">", "<=", ">="],
    "wrong return": ["return ", "wrong return"],
    "wrong assignment": ["=", "+=", "-=", "*=", "/="],
    "missing initialization": ["initialize", "initialization missing"],
    "extra statement": ["extra line", "unnecessary", "redundant"],
    "infinite loop": ["infinite loop", "while True", "no break"],
    "wrong function call": ["wrong call", "function call error"],
    "missing edge case": ["edge case missing", "boundary condition"],
    "logic bug": ["logic error", "logic bug"],
    "data structure misuse": ["list instead of dict", "wrong data structure"],
    "API misuse": ["API misuse", "wrong API call"],
    "syntax error": ["syntax error", "SyntaxError"],
}

def classify_bug(code_str):
    code_lower = code_str.lower()
    for bug, keywords in bug_classes.items():
        for kw in keywords:
            if kw in code_lower:
                return bug
    return "unknown"

# Step 6: Load and save code utils
def load_code(path):
    with open(path, "r") as f:
        return f.read()

def save_fixed_code(code_str, path):
    with open(path, "w") as f:
        f.write(code_str)

# Step 7: Extract python code from model output
def extract_code_from_model_output(output: str) -> str:
    match = re.search(r"```python\s*(.*?)```", output, re.DOTALL | re.IGNORECASE)
    if match:
        code = match.group(1)
    else:
        lines = output.split("\n")
        code_lines = []
        for line in lines:
            if line.strip() == "":
                if code_lines:
                    break
                else:
                    continue
            if re.match(r"^\s+|^[a-zA-Z_]", line):
                code_lines.append(line)
            else:
                if code_lines:
                    break
        code = "\n".join(code_lines)
    return code.strip()

# Step 8: Fix code using the LLM
def fix_code_with_model(code_str, temperature=0.1):
    prompt = (
        "You are an expert Python developer. The following Python code has a single-line bug:\n"
        "```\n"
        f"{code_str}\n"
        "```\n"
        "Please identify and fix the bug, preserving the original algorithm and style. Return ONLY the fixed code inside a ```python ... ``` code block."
    )
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=512,
        do_sample=True,
        temperature=temperature,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    fixed_code = extract_code_from_model_output(raw_output)
    return fixed_code

# Step 9: Run tests silently, returning test message only
def run_tests_on_fixed_code(code_str):
    temp_path = "temp_fixed.py"
    with open(temp_path, "w") as f:
        f.write(code_str)

    try:
        spec = importlib.util.spec_from_file_location("temp_fixed", temp_path)
        mod = importlib.util.module_from_spec(spec)
        sys.modules["temp_fixed"] = mod
        spec.loader.exec_module(mod)
    except Exception as e:
        return f"Import failed: {e}"

    try:
        tests = [getattr(mod, func) for func in dir(mod) if func.startswith("test_") and callable(getattr(mod, func))]
        if not tests:
            return "No test functions found."
        for test_func in tests:
            try:
                test_func()
            except Exception as e:
                return f"Test failed: {e}"
    except Exception as e:
        return f"Error running tests: {e}"

    return "All tests passed."

# Step 10: Main pipeline for Gradio UI
def pipeline_ui(file_selector, custom_code, temperature):
    if custom_code.strip():
        buggy_code = custom_code
    elif file_selector:
        buggy_code = load_code(file_selector)
    else:
        return "Error: Please select a file or enter code.", "", ""

    fixed_code = fix_code_with_model(buggy_code, temperature=temperature)
    test_message = run_tests_on_fixed_code(fixed_code)
    bug_type = classify_bug(buggy_code)

    return fixed_code, bug_type, test_message

# Step 11: List buggy python files from repo
buggy_dir = "Code-Refactoring-QuixBugs/python_programs"
python_files = [os.path.join(buggy_dir, f) for f in os.listdir(buggy_dir) if f.endswith(".py")]

# Step 12: Launch Gradio Blocks UI without flag button
with gr.Blocks() as demo:
    gr.Markdown("# QuixBugs Automated Code Correction")
    gr.Markdown(
        "Select a buggy Python file OR paste a single-line buggy Python code snippet. "
        "The system will fix the bug, classify it, and run available tests."
    )

    with gr.Row():
        file_selector = gr.Dropdown(choices=python_files, label="Select buggy Python file (optional)", interactive=True)
        custom_code = gr.Textbox(lines=5, placeholder="Or paste single-line buggy Python code here", label="Or enter custom buggy code")

    temperature = gr.Slider(minimum=0.0, maximum=1.0, value=0.1, label="Temperature (Lower = More deterministic)")

    submit_btn = gr.Button("Fix Bug")

    fixed_code_output = gr.Textbox(label="Fixed Code", lines=20)
    bug_type_output = gr.Textbox(label="Classified Bug Type")
    test_msg_output = gr.Textbox(label="Test Message")

    def run_pipeline(file_sel, code, temp):
        return pipeline_ui(file_sel, code, temp)

    submit_btn.click(run_pipeline, inputs=[file_selector, custom_code, temperature], outputs=[fixed_code_output, bug_type_output, test_msg_output])

demo.launch()


Repo already cloned.
Loading model (this may take a minute)...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://052254e5003e61647b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
